# Mountain Car
### Pierpaolo Spaziani - 0316331

In [60]:
import gym
import numpy as np
import imageio

In [61]:
# In the gym library there is a deprecation about NumPy
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module="gym.utils.passive_env_checker")

### Parameters

In [68]:
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EPISODES = 2000

DISCRETE_SIZE = 20

RENDER = False
SHOW_EVERY = 2

In [69]:
if RENDER:
    env = gym.make("MountainCar-v0", render_mode="rgb_array_list")
else:
    env = gym.make("MountainCar-v0")

### Discretization
In the presence of *continuous states* it is convenient to **discretize** for several reasons:
- Limit the state space
- Memory management
- Faster training
- Generalization

In [70]:
chunk_size = (env.observation_space.high - env.observation_space.low) / DISCRETE_SIZE

def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low) / chunk_size
    return tuple(discrete_state.astype(int))

### Q-Learning

Initialization of the Q-table and definition of the update algorithm:

In [74]:
q_table = np.zeros((DISCRETE_SIZE, DISCRETE_SIZE, env.action_space.n))

def q_learning():
    pass

In [72]:
def simulate():
    for episode in range(EPISODES):
        print("Episode:", episode, end="\r")
        
        env.reset()
    
        if (RENDER and episode % SHOW_EVERY == 0):
            render = True
        else:
            render = False
    
        discrete_state = get_discrete_state(env.state)
        
        done = False
        while not done:
    
            action = np.argmax(q_table[discrete_state])
            observation, reward, terminated, truncated, info = env.step(action)
            new_discrete_state = get_discrete_state(observation)
        
            done = terminated or truncated
            if not done:
                max_future_q = np.max(q_table[new_discrete_state])
                current_q = q_table[discrete_state + (action, )]
                new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
                q_table[discrete_state + (action, )] = new_q
            elif terminated:
                print(f"Congratulation! We reached to the goal! Episode: {episode}")
                q_table[discrete_state + (action, )] = 0 # ----------------------------------------> SECONDO ME QUESTO VA TOLTO
    
            discrete_state = new_discrete_state
            
        if render:
            imageio.mimsave(f'./gifs/{episode}.gif', env.render(), fps=30)
    
    env.close()

In [ ]:
simulate()